## Assignment Week 5

### Analysing the differences between the sexes by age in Ireland.

### Author: Gerry Callaghan



In [1]:
# I will want pandas to manipulate my data
import pandas as pd 
# I will want numpy for calculating the descriptive statistics
import numpy as np

# Reading in the Data
We will read in the data in a the form of a CSV file from the CSO.

In [2]:
# The data is sourced from the Central Statistics Office (CSO) of Ireland
# The dataset is titled "Population by Age Group and Sex"
# The data is available in CSV format via the CSO API
url = "https://ws.cso.ie/public/api.restful/PxStat.Data.Cube_API.ReadDataset/FY006A/CSV/1.0/en"

# read in the data from the csv file into a pandas DataFrame
df = pd.read_csv(url)

# lets take a look at the first few rows of the DataFrame
print(f"{df.head()}\n")


   STATISTIC Statistic Label  TLIST(A1)  CensusYear C02199V02655         Sex  \
0  FY006AC01      Population       2022        2022            -  Both sexes   
1  FY006AC01      Population       2022        2022            -  Both sexes   
2  FY006AC01      Population       2022        2022            -  Both sexes   
3  FY006AC01      Population       2022        2022            -  Both sexes   
4  FY006AC01      Population       2022        2022            -  Both sexes   

  C02076V03371 Single Year of Age                          C03789V04537  \
0            -           All ages                                   IE0   
1            -           All ages  2ae19629-1492-13a3-e055-000000000001   
2            -           All ages  2ae19629-1433-13a3-e055-000000000001   
3            -           All ages  2ae19629-149f-13a3-e055-000000000001   
4            -           All ages  2ae19629-14a0-13a3-e055-000000000001   

                 Administrative Counties    UNIT    VALUE  
0       

# Cleaning/Simplifying the data
While the data might be clean, there may be a lot of superfluous information in it that we can remove to simplify things.

In [3]:
# I only want data for Ireland as a whole - not all the administrative counties, 
# so I will filter the DataFrame to only include rows 
# where the Administrative Counties column is equal to "Ireland"

# I set my dataframe "df" equal to a dataframe comprising of data where for every row, 
# the entry for administrative county is equal to "Ireland"
df=df[df["Administrative Counties"]== "Ireland"]

# now i export out my dataframe to a csv file
df.to_csv("gender_population_for_analysis.csv")


In [4]:
# I can now view all the column headers to see what columns I can drop
headers = df.columns.tolist()
print(f"{headers}\n")

['STATISTIC', 'Statistic Label', 'TLIST(A1)', 'CensusYear', 'C02199V02655', 'Sex', 'C02076V03371', 'Single Year of Age', 'C03789V04537', 'Administrative Counties', 'UNIT', 'VALUE']



In [5]:
# so there are quite a few columns I don't need for my analysis, so I will drop them

# first I create a list of the columns to drop, 
# I can drop administrative counties now because I have already filterd the data for just Ireland
drop_col_list = ["STATISTIC","Statistic Label","TLIST(A1)","CensusYear","C02199V02655","C02076V03371","C03789V04537",'Administrative Counties', "UNIT"]

# then I can feed the list of columns to drop into the pandas drop() method, 
# "in place" means remove the data from the existing dataframe
# I could alternatively have created a new dataframe and set it equal to the columns that I only want to be left
df.drop(columns=drop_col_list, inplace=True)

# I will now view the column headers again to confirm the columns were dropped 
headers = df.columns.tolist()
print(f"{headers}\n")


['Sex', 'Single Year of Age', 'VALUE']



In [6]:
# I will have a quick look at the first few rows of the cleaned DataFrame
print(f"{df.head()}")
df.to_csv("gender_population_for_analysis.csv")  

            Sex Single Year of Age    VALUE
0    Both sexes           All ages  5149139
32   Both sexes       Under 1 year    57796
64   Both sexes             1 year    56420
96   Both sexes            2 years    59210
128  Both sexes            3 years    60484


In [7]:
# From my gender_population_for_analysis.csv file, I have three entries for Sex, 
# (both sexes, male, and female) but I only want male and female,
# so I will remove both sexes from the DataFrame 
df = df[df["Sex"]!="Both sexes"]

# just to see what the first few rows look like now
print(f"{df.head()}")

#now to view that my CSV file does not include both sexes entries, I export it again to a CSV to view
df.to_csv("gender_population_for_analysis.csv")  

       Sex Single Year of Age    VALUE
3264  Male           All ages  2544549
3296  Male       Under 1 year    29610
3328  Male             1 year    28875
3360  Male            2 years    30236
3392  Male            3 years    31001


In [8]:
## From my gender_population_for_analysis.csv file, 
# I have entries for "All ages" which I don't need for my analysis

# so I will remove all ages from the DataFrame 
df = df[df["Single Year of Age"]!="All ages"]

# just to see what the first few rows look like now
print(f"{df.head()}")

#now to view that my CSV file does not include all ages
df.to_csv("gender_population_for_analysis.csv")  

       Sex Single Year of Age  VALUE
3296  Male       Under 1 year  29610
3328  Male             1 year  28875
3360  Male            2 years  30236
3392  Male            3 years  31001
3424  Male            4 years  31686


# Reformating the Data
There are some entries in the data that makes it difficult to work with,
so let's simplify things

In [9]:
# I also don't want under 1 year of age, so I will convert that to 0 
df["Single Year of Age"] = df["Single Year of Age"].str.replace("Under 1 year","0")

# I also don't want over 100 year of age, so I will convert that to 100 
df["Single Year of Age"] = df["Single Year of Age"].str.replace("100 years and over","100")

# I also don't want the word "years", so I will remove text in that series altogether using regular expressions
df["Single Year of Age"] = df["Single Year of Age"].str.replace("\D", "", regex=True)


In [10]:
# now I can convert the Single Year of Age column to integer data type instead of type object
df["Single Year of Age"] = df["Single Year of Age"].astype("Int64")

# and likewise with the VALUE column instead of type object
df["VALUE"] = df["VALUE"].astype("Int64")

# I want to reorder the columns so I have Single Year of Age now the first column, 
# followed by Sex, followed by Number of people at that age
df_reordered = df.iloc[:, [1, 0, 2 ]]
# assign df equal to this reordered dataframe
df= df_reordered

# to view the top of my dataframe to confirm column order is correct
headers = df.columns.tolist()
print(f"{headers}\n")

# now to view that my CSV file shows cleaned age data
df.to_csv("gender_population_for_analysis.csv") 


['Single Year of Age', 'Sex', 'VALUE']



# Create a Pivot Table
Now that I have the data simplified down to Single Year of Age and Gender(s), 
and values each combination of age and gender, putting it in a table allows me to work with the data

In [11]:
# cross tables don't work because they count the number of people
#df_analysis = pd.crosstab(df.loc[:,"Single Year of Age"],df.loc[:,"Sex"])

# I will create a pivot table from the cleaned DataFrame for my analysis
df_analysis = pd.pivot_table(df, "VALUE","Single Year of Age","Sex")

# I will see what the first few rows of the analysis DataFrame look like
print(f"{df_analysis.head(5)}\n")

#now to view that my CSV file shows cleaned age data
df_analysis.to_csv("gender_population_for_analysis.csv")

# Note Single Year of Age is now my index, it is not a column,
# this is something I need to be wary of later when performing calculations between columns.


Sex                  Female     Male
Single Year of Age                  
0                   28186.0  29610.0
1                   27545.0  28875.0
2                   28974.0  30236.0
3                   29483.0  31001.0
4                   29819.0  31686.0




Normally, if I calculate the mean of a series of figures I just sum them and divide by the number of observations. But, in this case, in my population sample, the people are not evenly distributed across every single age.<br/>

I must weight each age by the number of people at that age. This is why I use the weighted mean.The same is the case for the weighted variance and standard deviation.

# Weighted Mean (Average)

The formula for calculating the weighted mean is as follows:<br/>
<img src="https:\\www.mathcracker.com\images\legacy\weighted-average-calculator.png" /><br/>

In my case, the weights (w) are the number of people at each age while the values (x) are all the ages. So, the weighted mean in my case is: sum (each age * female population at that age) / sum (female population)

In [12]:
# In my dataframe, I have ages (0-100) as my index and then two columns, female and male, 
# with the numbers of males and female at each age. I can confirm this as follows
headers = df_analysis.columns.tolist()
print(f"{headers}\n")

# Using summary statistics from the following command does not work
print(df_analysis["Male"].describe())
# because it just adds up the number of ages (101) and adds up the total number of males,
# and divides total number of males 2,544,549 by the total number of ages 101
# to give an average male age of 25,193.55 which makes no sense. 


['Female', 'Male']

count           101.0
mean     25193.554455
std      12752.169347
min             105.0
25%           17473.0
50%           30234.0
75%           34926.0
max           41644.0
Name: Male, dtype: Float64


In [13]:
# I need to get the weighted average, which takes into account the number of males and females at each age.
     
# Let's assume we take the first column, females at each age, in our dataframe, females are column 0.
gender = headers[0]
# We are setting our variable gender equal to all the values in the female column
# Using numpy, we give it the column containing the ages, that is, the index as X
# and we tell numpy that our weights w are the value in the column 0 of the dataframe df.
weighted_mean_female = np.average(df_analysis.index, weights=df_analysis[gender])
print(f"The weighted mean age of {gender}s is {round(weighted_mean_female,1)} years.\n")

# Let's assume we take the first column, males at each age, in our dataframe, females are column 0.
gender = headers[1]
weighted_mean_male = np.average(df_analysis.index, weights=df_analysis[gender])
print(f"The weighted mean age of {gender}s is {round(weighted_mean_male,1)} years.\n")

# Their averages are quite close, only one year between them.


The weighted mean age of Females is 38.9 years.

The weighted mean age of Males is 37.7 years.



# Weighted Median

While often confused with the mean, the median is what the mean would be if it was adjusted for certain values skewing the overall dataset. It does this by splitting the data equally in two in terms of numbers of observations, not size, and then focuses on what the mid-point observation is, it's one value if there is an odd number of observations because it's possible to split the data in even numbers of observations around that value.<br/><br/>
But, the median can be equal to the average of two values if there is an even number of observations because only by taking two numbers can there be an equal number of observations either side of those two values.

In [14]:
# First, I must sort the data. In my case, the data is already sorted by age (from 0 up to 100) 
# If it wasn't already sorted, we might sort it as follows.
df.sort_values('Single Year of Age', inplace=True)

# I then create a series called the cumulative sum, and incrementally add up all the values at each age
# Then I can simply halve that, and find the index of that middle value

#Let's assume I take the first column, female
gender = headers[0]
cumsum_female = df_analysis[gender].cumsum()
cutoff_female = df_analysis[gender].sum() / 2.0

# So, adding up all the observations at each age tells me there are 2,604,590 women
#print(f"The cumulative sum is: {cumsum_female}\n")
# and the woman in the middle is in the 1,302,295th position 
# (w3schools shows how to include the comma here https://www.w3schools.com/python/trypython.asp?filename=demo_fstring_modifier_comma)
# the :, add in the thousands comma while the .0f rounds off to the nearest whole number
print(f"The middle index for women is: {cutoff_female:,.0f}\n")

# To find where that woman in the 1,302,295th position is,
# starting at age 0, I add up the numbers of observations at each age until I reach the 1,302,295th woman.
# This happens where the cumulative sum is greater than or equal to the cutoff (middle index -> 1,302,295th woman)
# To find the index (age) of the first value where this is true 
# I take the dataframe, select the relevant gender column and then the row 
# where the cumulative sum which was calculated above is greater than the middle figure which we also calculated above.
median_female = df_analysis[gender][cumsum_female >= cutoff_female].index[0]
print(f"The weighted median age of {gender}s is {median_female}\n")

# Alternatively, because Single Year of Age is my index, I could reset it as a column
# df_analysis=df_analysis.reset_index()
# then find the age index at that 1,302,295th position as follows
#median = df_analysis["Single Year of Age"][cumsum_female >= cutoff_female].iloc[0]

# Now for the males, I just change the reference to the column to reference the second column
gender = headers[1]
cumsum_male = df_analysis[gender].cumsum()
cutoff_male = df_analysis[gender].sum() / 2.0
#print(f"The cumulative sum is: {cumsum_male}\n")
print(f"The middle index for men is: {cutoff_male:,.0f}\n")

median_male = df_analysis[gender][cumsum_male >= cutoff_male].index[0]

print(f"The weighted median age of {gender}s is {median_male}\n")

The middle index for women is: 1,302,295

The weighted median age of Females is 39

The middle index for men is: 1,272,274

The weighted median age of Males is 38



The median ages of males and females are very close to one another, and indeed are both very close to their respective means.<br/>
This is intuitive because it is unlikely that we would abnormally large numbers of observations at either end of the age scale.

## Weighted Mode

The mode is the most popular occurance of a variable. In this context, what age has the highest numbers of observations.<br/>
If we think about this, we are simply looking for the maximum value in the column, and then the age index that maximum occurs.


Note that I used the max() function in pandas to calculate the maximum of the relevant gender column. 
Then I went looking inside the dataframe df_analysis, under the relevant gender column,
until that part of the dataframe was equal to the highest_observation. In effect, I masked all other values as false.

Another alternative I found online was to use [idxmax()](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.idxmax.html) 
I showed how to use that also.

In [15]:
gender = headers[0]
# I use the built-in function max() to just get the maximum value in the respective gender columns
highest_observations= df_analysis[gender].max()
print(f"The highest number of observations in any one female age was: {highest_observations:,.0f}\n")

# let's look in the df_analysis dataframe, under the relevant gender column 
# where that maximum value occurs and the index ( that is, age) at that maximum value
mode_female = df_analysis[gender][df_analysis[gender]==highest_observations].index[0]
print(f"The weighted mode age of {gender} is {mode_female}\n")

# alternative method using idxmas() function from https://www.w3schools.com/python/pandas/ref_df_idxmax.asp
# Returns the index of the row with the highest males, and the index of that row
#mode_female = df_analysis[gender].idxmax()
#print(f"The weighted mode age of {gender} is {mode_female}\n")

# Same procedure fo the males
gender = headers[1]
highest_observations= df_analysis[gender].max()
print(f"The highest number of observations in any one male age was: {highest_observations:,.0f}\n")

# let's look in the df_analysis dataframe, under the relevant gender column where that value occurs
# and at what index ( that is, age)
mode_male = df_analysis[gender][df_analysis[gender]==highest_observations].index[0]
print(f"The weighted mode age of {gender} is {mode_male}\n")

# alternative method
# mode_male = df_analysis[gender].idxmax()
# print(f"The weighted mode age of {gender} is {mode_male}\n")


The highest number of observations in any one female age was: 43,696

The weighted mode age of Female is 41

The highest number of observations in any one male age was: 41,644

The weighted mode age of Male is 42



Similar to their means and medians, the most popular ages for men and women are close together, at 41 and 42.<br/>
Interestingly, the mode ages are relatively close to the media ages, which suggests relatively similar datasets to date.

# Weighted Variance

The variance says how dispersed the data is around the mean.<br/> 
The formula for calculating the weighted variance is as follows:<br/>
<img src="https://help.altair.com/panopticon/authoring/assets/images/weighted.png"/><br/>
I will simplify this to variance being the AVERAGE of the square of the differences from the mean


In [16]:
# I have already calculated the weighted means up above
# I can now simply use numpy to calculate the average of the squared difference from the mean
# then divide this by the number of that gender being observed

gender = headers[0]
weighted_variance_female = np.average((df_analysis.index - weighted_mean_female)**2, weights=df_analysis[gender])
print(f"The weighted variance of age for {gender}s is: {weighted_variance_female:,.0f}\n")

gender = headers[1]
weighted_variance_male = np.average((df_analysis.index - weighted_mean_male)**2, weights=df_analysis[gender])
print(f"The weighted variance of age for {gender}s is: {weighted_variance_male:,.0f}\n")

The weighted variance of age for Females is: 529

The weighted variance of age for Males is: 514



While the variance for females is greater than that of men, it is not by much, which definitely tells us how similar their datasets are.

# Weighted Standard Deviation

One issue with variance is that its size is not denominated in the same terms as the underlying observations. This is attributable to us squaring the differentials in its calculation. Consequently, to bring it back into similar terms, we must calculate its square root, giving us a figure that we know as the standard deviation.

In [17]:
# we have already calculated our variances up above
# We can now simply use numpy to calculate the square roots of the variances

gender = headers[0]
weighted_stddev_female = np.sqrt(weighted_variance_female)
print(f"The weighted standard deviation of age for {gender} is: {weighted_stddev_female:,.0f}\n")

gender = headers[1]
weighted_stddev_male = np.sqrt(weighted_variance_male)
print(f"The weighted standard deviation of age for {gender} is: {weighted_stddev_male:,.0f}\n")

The weighted standard deviation of age for Female is: 23

The weighted standard deviation of age for Male is: 23



Given how relatively similar their variances were, we expect the standard deviation of male and female ages to be similar. With a standard deviation of 22 approximately, and mean ages of 38 approximately, we can say that 95% of the population (2 standard deviations) lie in the age group of 0 to 80 which is perhaps expected given that the average life expectancy would be in the 70s.

# Part 2 

I'm going to make a variable that stores an age (say 35).

I will group the people within 5 years of that age together, into one age group 

- maybe make 35 the index, then use iloc of above and below that age.

I will calculate the population difference between the sexes in that age group.

- sum up the total males and female in those rows

In [18]:
target_age = 35
# my minimum age will be
lower_target = target_age - 5
# my maximum age will be
upper_target = target_age + 6

# now i set a new dataframe equal to a range of 5 below and 6 above the target age
df_population_subset = df_analysis.iloc[lower_target:upper_target, :]
print(f"{df_population_subset}")

Sex                  Female     Male
Single Year of Age                  
30                  32841.0  30858.0
31                  33710.0  32237.0
32                  34382.0  32413.0
33                  34489.0  31888.0
34                  36284.0  33121.0
35                  37940.0  34695.0
36                  39030.0  35828.0
37                  39193.0  36427.0
38                  40902.0  37513.0
39                  42592.0  38749.0
40                  43143.0  40301.0


# Calculating the sum of the smaller sample
Now I can simply sum up the value in each column using the sum () function then subtract one from the other.

In [19]:

# sum up the females

# let gender equal the female column
gender = headers[0]
# use the sum() function to sum up all the women in this column of this new dataframe
total_number_female = df_population_subset[gender].sum()
print(f"The total amount of {gender}s in this subset is: {total_number_female:,.0f}\n")

# sum up the males

# let gender equal the male column
gender = headers[1]

# use the sum() function to sum up all the men in this column of this new dataframe
total_number_male = df_population_subset[gender].sum()
print(f"The total amount of {gender}s in this subset is: {total_number_male:,.0f}\n")

# Now to calculate the population difference
if total_number_female > total_number_male:
    print(f"There are {(total_number_female - total_number_male):,.0f} more females than males in this subsample.")
else:
    print(f"There are {(total_number_male - total_number_female):,.0f} more males than females in this subsample.")


The total amount of Females in this subset is: 414,506

The total amount of Males in this subset is: 384,030

There are 30,476 more females than males in this subsample.


# Part 3

I will now work out which region in Ireland has the biggest population difference between the sexes in that age group

The biggest population difference could be interpreted different ways, it could be measured by which has the greatest dispersion (variance/standard deviation) or it could be interpreted by which has the greatest difference between its mean values for each gender at each age in each region. The latter is the interpretation that I'm going to use. 


In [20]:
# I'm going to pull in the full csv file from the CSO again.

#  this is the URL
url = "https://ws.cso.ie/public/api.restful/PxStat.Data.Cube_API.ReadDataset/FY006A/CSV/1.0/en"

# read in the data from the csv file into a pandas DataFrame called df_by_counties to distinguish it from the earlier dataframes.
df_by_counties = pd.read_csv(url)


Now I need to clean up my dataframe, using similar methods to what I did earlier.

In [21]:
# first, I need to remove some superfluous columns again, so applying a similar method as before
drop_col_list = ["STATISTIC","Statistic Label","TLIST(A1)","CensusYear","C02199V02655","C02076V03371","C03789V04537", "UNIT"]
df_by_counties.drop(columns=drop_col_list, inplace=True)

In [22]:
# The opposite to earlier, now I only want every administrative region BUT Ireland
df_by_counties=df_by_counties[df_by_counties["Administrative Counties"]!= "Ireland"]

# Like before, i don't want both sexes
df_by_counties = df_by_counties[df_by_counties["Sex"]!="Both sexes"]

# I also don't want all ages as before
df_by_counties = df_by_counties[df_by_counties["Single Year of Age"]!="All ages"]

# For the Single Year of Age column, I'm going to clean it up exactly as I did above
df_by_counties["Single Year of Age"] = df_by_counties["Single Year of Age"].str.replace("Under 1 year","0")
df_by_counties["Single Year of Age"] = df_by_counties["Single Year of Age"].str.replace("100 years and over","100")
df_by_counties["Single Year of Age"] = df_by_counties["Single Year of Age"].str.replace("\D", "", regex=True)

# Now to change the column type of data from object to integer, as I did above also
df_by_counties["Single Year of Age"] = df_by_counties["Single Year of Age"].astype("Int64")
df_by_counties["VALUE"] = df_by_counties["VALUE"].astype("Int64")

In [23]:
# now i only want the target age group 

# this is my target age
target_age = 35
# my minimum age will be 5 years below that (31-35 incl)
lower_target = target_age - 5

# my maximum age will be 6 years above that (36-40 incl - remember I'm using the less than sign so it will not include 41)
upper_target = target_age + 6

df_by_counties_subset = df_by_counties[df_by_counties["Single Year of Age"]>lower_target] 
df_by_counties_subset = df_by_counties_subset[df_by_counties_subset["Single Year of Age"]<upper_target]             

# now i set a new dataframe equal to a range of 5 below and 6 above the target age
#df_by_counties_subset = [df_by_counties]["Single Year of Age"].iloc[lower_target:upper_target, :]
print(f"{df_by_counties_subset}")

         Sex  Single Year of Age                Administrative Counties  VALUE
4289    Male                  31                  Carlow County Council    349
4290    Male                  31                    Dublin City Council   6287
4291    Male                  31  Dún Laoghaire Rathdown County Council   1510
4292    Male                  31                  Fingal County Council   2061
4293    Male                  31            South Dublin County Council   1898
...      ...                 ...                                    ...    ...
7867  Female                  40               Roscommon County Council    556
7868  Female                  40                   Sligo County Council    538
7869  Female                  40                   Cavan County Council    630
7870  Female                  40                 Donegal County Council   1293
7871  Female                  40                Monaghan County Council    518

[620 rows x 4 columns]


In [24]:
# now i export out my dataframe to a csv file to see what i have
df_by_counties_subset.to_csv("gender_population_for_analysis_by_county.csv")

Now, I will split my sample in two, one for male, and one for female. Then I can create a pivot table from the remaining variables, counties against age.

In [25]:
df_by_counties_subset_male = df_by_counties_subset[df_by_counties_subset["Sex"] =="Male"]
print(f"{df_by_counties_subset_male}\n")

df_by_counties_subset_female = df_by_counties_subset[df_by_counties_subset["Sex"] =="Female"]
print(f"{df_by_counties_subset_female}")

       Sex  Single Year of Age                Administrative Counties  VALUE
4289  Male                  31                  Carlow County Council    349
4290  Male                  31                    Dublin City Council   6287
4291  Male                  31  Dún Laoghaire Rathdown County Council   1510
4292  Male                  31                  Fingal County Council   2061
4293  Male                  31            South Dublin County Council   1898
...    ...                 ...                                    ...    ...
4603  Male                  40               Roscommon County Council    489
4604  Male                  40                   Sligo County Council    491
4605  Male                  40                   Cavan County Council    633
4606  Male                  40                 Donegal County Council   1149
4607  Male                  40                Monaghan County Council    477

[310 rows x 4 columns]

         Sex  Single Year of Age                Adm

In [26]:
#Now that I have two separate dataframes, one for each sex

# I will create a pivot table from each of the cleaned DataFrame for my analysis
# male
df_by_counties_subset_male_analysis = pd.pivot_table(df_by_counties_subset_male, "VALUE","Single Year of Age","Administrative Counties")
# female
df_by_counties_subset_female_analysis = pd.pivot_table(df_by_counties_subset_female, "VALUE","Single Year of Age","Administrative Counties")

# I will see what the first few rows of the analysis DataFrame look like
# male
print(f"{df_by_counties_subset_male_analysis.head(5)}\n")
# female
print(f"{df_by_counties_subset_female_analysis.head(5)}\n")

# now to view that my CSV file shows cleaned age data
# male
df_by_counties_subset_male_analysis.to_csv("gender_population_for_analysis_by_county_for_males.csv")
# female
df_by_counties_subset_female_analysis.to_csv("gender_population_for_analysis_by_county_for_females.csv")


Administrative Counties  Carlow County Council  Cavan County Council  \
Single Year of Age                                                     
31                                       349.0                 450.0   
32                                       362.0                 466.0   
33                                       366.0                 466.0   
34                                       360.0                 475.0   
35                                       387.0                 519.0   

Administrative Counties  Clare County Council  Cork City Council  \
Single Year of Age                                                 
31                                      623.0             1767.0   
32                                      642.0             1727.0   
33                                      643.0             1633.0   
34                                      714.0             1636.0   
35                                      729.0             1687.0   

Administrative Cou

In [27]:
# I'm now going to calcutale the weighted mean, variance, and standard deviation for each county.

# this variable will be the running value of the greatest difference in the means between the gender for each county
max_difference = 0

# I will circle through each county printing out their summary statistics in turn
for county in df_by_counties_subset_female_analysis:


    # the weighted mean, variance, and standard deviation calculations are the same as previous, except now we have a smaller sample
    print(f"The statistics for {county} are as follows:\n")

    # Mean
    weighted_mean_female_by_counties_subset = np.average(df_by_counties_subset_female_analysis.index, weights=df_by_counties_subset_female_analysis[county])
    print(f"The weighted mean age for women in this age group is:\t{(weighted_mean_female_by_counties_subset):,.2f} years.")
    weighted_mean_male_by_counties_subset = np.average(df_by_counties_subset_male_analysis.index, weights=df_by_counties_subset_male_analysis[county])
    print(f"The weighted mean age for men in this age group is:\t{(weighted_mean_male_by_counties_subset):,.2f} years.")

    # Variance
    weighted_variance_female_by_counties_subset = np.average((df_by_counties_subset_female_analysis.index - weighted_mean_female_by_counties_subset)**2, weights=df_by_counties_subset_female_analysis[county])
    print(f"The weighted variance for women in this age group is:\t{(weighted_variance_female_by_counties_subset):,.2f}")
    weighted_variance_male_by_counties_subset = np.average((df_by_counties_subset_male_analysis.index - weighted_mean_male_by_counties_subset)**2, weights=df_by_counties_subset_male_analysis[county])
    print(f"The weighted variance for men in this age group is:\t{(weighted_variance_male_by_counties_subset):,.2f}")
    
    # Standard deviation
    weighted_stddev_female_by_counties_subset = np.sqrt(weighted_variance_female_by_counties_subset)
    print(f"The weighted standard deviation for women in this age group is:\t{(weighted_stddev_female_by_counties_subset):,.2f} years.")
    weighted_stddev_male_by_counties_subset = np.sqrt(weighted_variance_male_by_counties_subset)
    print(f"The weighted standard deviation for men in this age group is:\t{(weighted_stddev_male_by_counties_subset):,.2f} years.\n")

    # so if the current county has a greater difference in its mean than the current highest, it becomes the highest
    new_max_difference = abs(weighted_mean_female_by_counties_subset-weighted_mean_male_by_counties_subset)
    if new_max_difference > max_difference:
        max_difference = new_max_difference
        county_highest_difference = county
    

print(f"\nThe county with the greatest population difference is {county_highest_difference} with a difference of {new_max_difference:,.2f} years in the mean ages.")






The statistics for Carlow County Council are as follows:

The weighted mean age for women in this age group is:	35.80 years.
The weighted mean age for men in this age group is:	35.83 years.
The weighted variance for women in this age group is:	8.21
The weighted variance for men in this age group is:	8.28
The weighted standard deviation for women in this age group is:	2.87 years.
The weighted standard deviation for men in this age group is:	2.88 years.

The statistics for Cavan County Council are as follows:

The weighted mean age for women in this age group is:	35.83 years.
The weighted mean age for men in this age group is:	35.85 years.
The weighted variance for women in this age group is:	8.05
The weighted variance for men in this age group is:	8.21
The weighted standard deviation for women in this age group is:	2.84 years.
The weighted standard deviation for men in this age group is:	2.87 years.

The statistics for Clare County Council are as follows:

The weighted mean age for wome